<a href="https://colab.research.google.com/github/DODO-YONG/Study/blob/main/3w_glove%26fastext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install glove_python_binary

     |████████████████████████████████| 948 kB 8.3 MB/s 


In [3]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7efc987d6e90>)

In [8]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [9]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [10]:
print(glove.most_similar("man"))

[('woman', 0.9560786418903613), ('guy', 0.8775509100121951), ('girl', 0.876918931684838), ('kid', 0.8364047136342387)]


In [11]:
print(glove.most_similar("boy"))

[('girl', 0.9375457608695562), ('woman', 0.853146726388624), ('man', 0.8255367850492633), ('kid', 0.8212116881928796)]


In [12]:
print(glove.most_similar("clean"))

[('fresh', 0.8204097329607798), ('water', 0.8128160738293824), ('heat', 0.8106721854240201), ('wind', 0.7959817776990797)]


# Fast Text

내부 단어(subword)의 학습

모르는 단어(Out Of Vocabulary, OOV)에 대한 대응

단어 집합 내 빈도 수가 적었던 단어(Rare Word)에 대한 대응

1) Word2Vec

In [13]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [14]:
# w2v의 경우 모르는 단어는 임베딩 벡터가 존재하지 않기에 계산 불가
model_result = model.wv.most_similar("electrofishing")
print(model_result)

KeyError: ignored

2) FastText

모르는 단어 가능함

In [15]:
from gensim.models import FastText

model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [16]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.7973754405975342),
 ('electro', 0.7868226766586304),
 ('electrolyte', 0.785539448261261),
 ('electric', 0.7653621435165405),
 ('fishing', 0.7615876793861389),
 ('electroshock', 0.7595863938331604),
 ('electronic', 0.7531237602233887),
 ('electrons', 0.7519485950469971),
 ('gastric', 0.7496391534805298),
 ('electrochemical', 0.7489936351776123)]